In [98]:
import logging
import os
import codecs,sys

import jieba
import string
import re

import json
import pandas as pd

In [56]:
logging.basicConfig(format='[%(levelname)s] %(message)s',
                   level=logging.DEBUG)
logger = logging.getLogger()

In [57]:
def read_data(file_dir):
    contents = []
    files = os.listdir(file_dir)
    for file in files:
        #print(os.path.join(file_dir,file))
        with codecs.open(os.path.join(file_dir,file),'r',
                         encoding='gbk',errors='ignore') as fr:
            content = fr.readlines()
            for c in content:
                c = c.strip()
                if len(c)!=0:
                    contents.append(c)
                
    return contents

In [58]:
neg_dir = './ChnSentiCorp_htl_ba_2000/neg'
pos_dir = './ChnSentiCorp_htl_ba_2000/pos'
neg_data = read_data(neg_dir)
pos_data = read_data(pos_dir)

In [67]:
def clearData(data):
    new_data = []
    for line in data:
        line = line.strip()
        intab, outtab = "",""
        transtab = str.maketrans(intab,outtab)
#         print(transtab)
        pun_num = string.punctuation + string.digits
#         print(pun_num)
#         line = line.encode('utf-8')
#         line = line.translate(transtab,pun_num)
#         line = line.decode('utf-8')
        line = re.sub("[a-zA-Z0-9]","",line)
        line = re.sub("[\s+\.\!\/_,$%^*(+\"\'；：“”．]+|[+——！，。？?、~@#￥%……&*（）]+"
                     ,"",line)
        new_data.append(line)
    return new_data

In [71]:
def sent2word(data):
    new_data = []
    for line in data:
        segList = jieba.cut(line,cut_all=False)
        segSentence = ''
        for word in segList:
            if word != '\t':
                segSentence += word + " "
        new_data.append(segSentence.strip())
    return new_data

In [95]:
stopCount = [w.strip() for w in codecs.open('stopWord.txt','r',
                                            encoding='utf-8',errors='ignore').readlines()]
# print(stopCount)
def removeStop(data):
    new_data = []
    sentence = ''
    for line in data:
        tmp = []
        line = line.split(' ')
        for word in line:
            if word not in stopCount:
                tmp.append(word)
        new_data.append(tmp)
    return new_data

In [111]:
#print(neg_data[0])
neg_new = clearData(neg_data)
pos_new = clearData(pos_data)
#print(neg_new[0])
neg_new_clip = sent2word(neg_new)
pos_new_clip = sent2word(pos_new)
# print(neg_new_clip[:3])
neg_new_clip_sop = removeStop(neg_new_clip)
pos_new_clip_sop = removeStop(pos_new_clip)
# print(neg_new_clip_sop[:3])

In [112]:
df = pd.DataFrame(columns=('id','content','label'))
for i in range(len(pos_new_clip_sop)):
    cont = pos_new_clip_sop[i]
#     print(cont)
    li = 1
    df = df.append(
        {
            'id':i,
            'content':cont,
            'label':li,
            
        },ignore_index=True 
    )
df.to_csv('data.csv',mode='a+',index=False,header=None)